# Exploration in Linear Bandits

The objective of this part is to implement and compare the following strategies for linear bandits:

[Optimism in the Face of Uncertainty (LinUCB/OFUL)](https://papers.nips.cc/paper/4417-improved-algorithms-for-linear-stochastic-bandits.pdf)

[Thompson Sampling](https://projecteuclid.org/euclid.ejs/1513306870)

## Linear bandit
We consider the standard linear bandit setting. At each time $t$, the agent selects an arm $a_t \in A$ and observes a reward
$$
r_{a}^t = \langle \theta^\star, \phi_a^t \rangle + \eta_a^t := \mu_a^t + \eta_a^t
$$
where $\theta^{\star} \in \mathbb{R}^{d}$ is a parameter vector, $\phi_{a}^t \in \mathbb{R}^{d} $ are the features of arm $a$ at time $t$, and $\eta_{a}^{t}$ is a zero-mean  $\sigma^2$-subgaussian noise. 

When the features correspond to the canonical basis, this formulation reduces to multi-armed bandit (MAB) with $d$ arms. In the more general case, the features may depend on a context $x_t$, so that $\phi_a^t = \phi(x_t, a)$ denotes the feature vector of a context-action pair $(x_t, a)$ and the resulting setting is the so-called linear contextual bandit.

We rely on the following standard assumption on the features and the unknown parameter $\theta^\star$.

**Assumption.** There exist $B,D \geq 0$, such that $\|\theta^\star\|_2 \leq B$, $\|\phi_a^t\| \leq D$, and $\langle \theta^\star, \phi_a^t \rangle \in [0,1]$, for all $t$ and $a$.

Given a finite horizon $n$, the performance of the agent is measured by its (pseudo)-\emph{regret}:
$$
        %R(n) = n \mu^{\star} - \sum_{t=1}^n \mu_{a_t} = \sum_{i=1}^K T_i(n) \Delta_i, 
        R(n) = \sum_{t=1}^n \langle \theta^\star, \phi_{a^\star}^t \rangle - \langle \theta^\star, \phi_{a_t}^t \rangle ,
$$
where $a^{\star}_{t} \in \arg\max_{a} \langle \theta^\star, \phi_{a}^t \rangle$ is the optimal action at time $t$.

**We consider the simple linear bandit setting:** $\phi_a^t = \phi_a, \; \forall t$




In [0]:
!rm -rf mvarl_hands_on/
!git clone https://github.com/rlgammazero/mvarl_hands_on.git
!cd mvarl_hands_on/ && git fetch
!ls mvarl_hands_on/utils

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import os
import numpy as np
from pprint import pprint
from coldstart import ColdStartFromDataset
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import math
from tqdm import tqdm

#### Jester Jokes Dataset (Dense subset of 40 jokes)

Deep Bayesian Bandits Showdown: An Empirical Comparison of Bayesian Deep Networks for Thompson Sampling

Download the data at: https://storage.googleapis.com/bandits_datasets/jester_data_40jokes_19181users.npy

We performed a matrix factorization of the ratings (after filtering over users and jokes). This provides features for the arms and users, the reward (ie rating) is the dot product between the arm and user features (we make it stochastic by adding Gaussian noise). We consider a cold start problem where the user is randomly selected at the beginning of the repetition and the agent has to learn the best arm to recommend. When an arm is selected by the algorithm, its reward is computed as the dot product between the arm and user features.

In [0]:
!wget https://storage.googleapis.com/bandits_datasets/jester_data_40jokes_19181users.npy

In [0]:
M = np.load('jester_data_40jokes_19181users.npy')
M = M / 10
K = 35
U, s, Vt = svds(M, k=K)
s = np.diag(s)
U = np.dot(U,s)
print('U: {}'.format(U.shape))
print('Vt: {}'.format(Vt.shape))
print('#features: {}'.format(Vt.shape[0]))
print('#arms: {}'.format(Vt.shape[1]))
np.savetxt('U_jester.csv', U, delimiter=',')
np.savetxt('Vt_jester.csv', Vt, delimiter=',')

Create the coldstart model

In [0]:
seed = 1235
user_subset = np.linspace(0, 400, 10).astype(int).tolist()
arm_csvfile = os.path.abspath('Vt_jester.csv')
user_csvfile = os.path.abspath('U_jester.csv')
noise_std = 0.1

config_cs = {
    'arm_csvfile': arm_csvfile,
    'user_csvfile': user_csvfile,
    'random_state': seed,
    'user_subset': user_subset,
    'noise_std': noise_std
}

print("Current config is:")
pprint(config_cs)

In [0]:
noise = 0.2
random_state = 312
model = ColdStartFromDataset(**config_cs)
print("\nThe new user arriving to the system is user #", model.theta_idx)
print("\nTheta*: ", model.theta)
means = np.dot(model.features, model.theta)
print("\nMeans: ", means)
theta_bound = np.linalg.norm(model.theta, 2)
print("\nTheta bound: ", theta_bound)

**Question 1**: implement LinUCB

### LinUCB / OFUL
See the slides!

Note that it is not necessary to invert the matrix $A_t$ at each round. Since $A_t$ is obtained from a rank-1 update of $A_{t-1}$, it is possible to use Sherman–Morrison formula to build directly $A_t^{-1}$.

Suppose $𝐴$ be a nonsingular $n\times n$ matrix and $\mathbf{u}, \mathbf{v}$ be vectors. Then
$$
(A+\mathbf{u}\mathbf{v}^T)^{-1} = A^{-1} - \frac{A^{-1}\mathbf{u}\mathbf{v}^TA^{-1}}{1+\mathbf{v}^TA^{-1}\mathbf{u}}.
$$

In [0]:
class OFUL:
    def __init__(self, arm_features, reg_factor, delta,
                 bound_theta, noise_std):
        self.arm_features = arm_features
        self.reg_factor = reg_factor
        self.delta = delta
        self.iteration = 1
        self.bound_theta = bound_theta
        self.bound_features = np.max(np.sqrt(np.sum(np.abs(arm_features) ** 2, axis=1)))
        self.noise_std = noise_std

        self.theta = np.zeros(self.n_features)
        self.A = self.reg_factor * np.eye(self.n_features)
        self.inv_A = np.eye(self.n_features) / self.reg_factor
        self.b = np.zeros(self.n_features)

    @property
    def n_actions(self):
        return self.arm_features.shape[0]

    @property
    def n_features(self):
        return self.arm_features.shape[1]
    
    def compute_estimates(self):
        """Return the internal estimates
        """
        return self.arm_features @ self.theta

    def alpha(self, n_samples):
        """Return the alpha used to measure uncertainty
        """
        B = self.noise_std
        d = self.n_features
        t = n_samples # self.iteration
        L = self.bound_features
        lamb = self.reg_factor
        delta = self.delta

        # see slide 49
        alpha = B * np.sqrt(d * np.log((1 + t * L / lamb) / delta)) + np.sqrt(lamb) * self.bound_theta

        return alpha
    
    def sample_action(self):
        """Return the action to play based on current estimates
        """

        estimates = self.compute_estimates()
        uncertainty = np.zeros(estimates.shape)

        for k in range(len(uncertainty)):
          uncertainty[k] = self.alpha(self.iteration) * np.sqrt(self.arm_features[k].T @ self.inv_A @ self.arm_features[k])

        # see slide 50
        combined_value = estimates + uncertainty
        action = np.argmax(combined_value)

        return action
    
    def update(self, a_t, r_t):
        """Update the estimates of the model
        

        Parameters
        ----------
        a_t: int
            The action played at the current episode
        r_t: float
            The reward associated to action a_t

        Returns
        -------
        none
        """

        self.iteration += 1

        phi_at = self.arm_features[a_t, :]
        update = phi_at.reshape(-1, 1) @ phi_at.reshape(1, -1)

        self.A += update
        self.inv_A -= self.inv_A @ update @ self.inv_A / (1 + phi_at.T @ self.inv_A @ phi_at)
        self.b += r_t * phi_at
        self.theta = self.inv_A @ self.b

**Question 2:** implement LinearTS

Let $A_t$ be the design matrix, $\theta_t$ the estimate of $\theta^\star$ and $\beta_t$ the confidence interval built by LinUCB. Then, at every time step t,  LinearTS simply generates
a sample $\tilde{\theta}_t$ from the distribution $\mathcal{N}(\widehat{\theta}_t, \omega_t \alpha_t^2 A_t^{-1})$.

LinearTS

For $t=1, \ldots, T$
> $\tilde{\theta}_t \sim \mathcal{N}(\widehat{\theta}_t, \omega_t \alpha_t^2 A_t^{-1})$
>
> $a_t \in \arg\max_{a \in \mathcal{A}_t}  \langle \tilde{\theta}_t, \phi_{a} \rangle$
>
> observe reward $r_t$

TS is requires to draw $\tilde{\theta}_t$ from a distribution over-sampling by a factor $\sqrt{d}$ the ellipsoid constructed by OFUL (i.e., $\omega_t = d$). This is required to prove that LinearTS is optimistic with a fix probability. This is necessary to prove the frequentist regret of TS. The regret of TS is worse than the one of LinUCB by a factor $\sqrt{d}$ (i.e., $\widetilde{O}(d^{3/2}\sqrt{T})$)

In [0]:
class LinearTS(OFUL):
    def __init__(self, arm_features, reg_factor, delta,
                 bound_theta, noise_std, over_sample):
        self.over_sample = over_sample
        super(LinearTS, self).__init__(arm_features, reg_factor, delta,
                 bound_theta, noise_std)
        
    def compute_estimates(self):
        """Return the internal estimates
        """
        
        mean = self.theta
        cov = self.alpha(self.iteration) ** 2 * self.inv_A

        if self.over_sample:
          cov *= self.n_features

        theta_tilde = np.random.multivariate_normal(mean, cov)

        return self.arm_features @ theta_tilde

    def sample_action(self):
        """Return the action to play based on current estimates
        """

        return np.argmax(self.compute_estimates())

**Question 3**: run the algorithms (`LinUCB` and `LinearTS`) and average the performance over multiple users (ie simulations)

The regret $R(T) = \sum_t \phi_t^\top (\theta^\star - \theta_t)$

The performance is the expected regret over multiple users. You can also test `LinearTS` without the additional $\sqrt{d}$.

You can use `RandomLinearArms` to test your code before using Jester

In [0]:
# from mvarl_hands_on.utils.coldstart import RandomLinearArms

def compute_regrets(algorithms, nb_simulations, T, regret_formula='2'):
  """Compute the regrets
  """

  regrets = {}

  for alg_name in algorithms.keys():
      if alg_name not in regrets.keys():
          regrets[alg_name] = np.zeros((nb_simulations, T))
      
      for k in range(nb_simulations):
          if k % 1 == 0:
              print("{} simulation {}/{}".format(alg_name, k+1, nb_simulations))
          model = ColdStartFromDataset(**config_cs)
  #         model = RandomLinearArms()
          alg = algorithms[alg_name](arm_features=model.features, bound_theta=np.linalg.norm(model.theta))

          for t in range(T):
            a_t = alg.sample_action()
            r_t = model.reward(a_t)
            alg.update(a_t, r_t)

            # compute regret
            if regret_formula == '1':  # see formula in the second cell of the notebook
              best_action = np.argmax(model.features @ model.theta)
              regrets[alg_name][k, t] = model.theta @ (model.features[best_action, :] - model.features[a_t, :])
            
            elif regret_formula == '2':  # see formula of the question 3
              regrets[alg_name][k, t] = model.features[a_t, :] @ (model.theta - alg.theta)
  
  return regrets

def plot_regrets(regrets):
  """Plot the regrets
  """
  
  for alg_name in regrets.keys():
      data = np.cumsum(regrets[alg_name], axis=1)
      n_rep, T = data.shape
      
      mean_regret = np.mean(data, axis=0)
      std_regret = np.std(data, axis=0) / math.sqrt(n_rep)
      t = np.arange(T)
      plt.plot(t, mean_regret, label=alg_name)
      plt.fill_between(t, mean_regret - 2 * std_regret, mean_regret + 2 * std_regret, alpha=0.15)
  plt.legend()

The regret here is defined as: $R(n) = \sum_{t=1}^n \langle \theta^\star, \phi_{a^\star_t}^t \rangle - \langle \theta^\star, \phi_{a_t}^t \rangle ,
$ where $a^{\star}_{t} \in \arg\max_{a} \langle \theta^\star, \phi_{a}^t \rangle$ is the optimal action at time $t$.

In [0]:
nb_simulations = 4
T = int(4e4)
regret_formula = '1'
print('Computing (pseudo)-regrets with the first formula of this notebook (second cell).\n')

algorithms = {
            'OFUL': lambda arm_features, bound_theta: 
              OFUL(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std']),
             'LinearTS': lambda arm_features, bound_theta:
              LinearTS(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std'], over_sample=False)
             }
regrets1 = compute_regrets(algorithms, nb_simulations, T, regret_formula)

algorithms = {
            'OFUL': lambda arm_features, bound_theta: 
              OFUL(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std']),
             'LinearTS': lambda arm_features, bound_theta:
              LinearTS(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std'], over_sample=True)
             }
regrets1_oversampling = compute_regrets(algorithms, nb_simulations, T, regret_formula)

plt.figure(figsize=(20,8))

plt.subplot(121)
plt.title('(Pseudo)-regrets without oversampling')
plot_regrets(regrets1)

plt.subplot(122)
plt.title('(Pseudo)-regrets with oversampling')
plot_regrets(regrets1_oversampling)

The regret used here is defined as: $R(T) = \sum_t \phi_t^\top (\theta^\star - \theta_t)$.

In [0]:
nb_simulations = 4
T = int(4e4)
regret_formula = '2'
print('Computing regrets with the second formula of this notebook (question 3).\n')

algorithms = {
            'OFUL': lambda arm_features, bound_theta: 
              OFUL(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std']),
             'LinearTS': lambda arm_features, bound_theta:
              LinearTS(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std'], over_sample=False)
             }
regrets2 = compute_regrets(algorithms, nb_simulations, T, regret_formula)

algorithms = {
            'OFUL': lambda arm_features, bound_theta: 
              OFUL(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std']),
             'LinearTS': lambda arm_features, bound_theta:
              LinearTS(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std'], over_sample=True)
             }
regrets2_oversampling = compute_regrets(algorithms, nb_simulations, T, regret_formula)

plt.figure(figsize=(20,8))

plt.subplot(121)
plt.title('Regrets without oversampling')
plot_regrets(regrets2)

plt.subplot(122)
plt.title('Regrets with oversampling')
plot_regrets(regrets2_oversampling)